# Prepare the datasets

## data449_text.csv

We use the data set data449_cats which is directly extracted from Label Studio with the csv format. It is made out of the labellisation of data. Our objective is to reduce it to two columns with the text and 0 labels.

In [ ]:
import pandas as pd 

# Read in the data
df = pd.read_csv('../data/raw/data449_cats.csv')

# Drop columns
df = df[["text"]].copy()

# Save to csv
df.to_csv('../data/processed/data449_text.csv', index=False)

## full_data_link_legifrance_id.xlsx

In [ ]:
import pandas as pd

df = pd.read_excel(r"../data/raw/full_data_link_legifrance.xlsx")
df = df[["UrlLegifrance", "Entreprise","Siret", "Naf732", "Date Texte","Fichier"]].copy()
df["code"] = df["Fichier"].str.slice(101,113)
df.to_csv('../data/processed/full_data_link_legifrance_id.csv', index=False)
df.to_excel('../data/processed/full_data_link_legifrance_id.xlsx', index=False)

## long_short_texts.csv

In [ ]:
import pandas as pd
import glob

We import the shorten texts

In [ ]:
# Read in the data
df_short_text = pd.read_excel(r'../data/raw/short_texts.xlsx').loc[0:724, ["code","entreprise","text"]]
df_short_text["length"] = df_short_text["text"].str.len()

mask = df_short_text["length"] > 2800

if mask.any():
    # Filter and keep only rows with text length less than or equal to 2800 characters
    df_short_text = df_short_text[~mask]

df_short_text = df_short_text[["code","entreprise","text"]].copy()

In [ ]:
df_short_text.shape

We build the long text database

In [ ]:
df_id = pd.read_csv(r'../data/processed/full_data_link_legifrance_id.csv')

df_text_link= pd.merge(df_id, df_short_text, on="code", how="inner")

In [ ]:
df_text_link = df_text_link[["code", "entreprise", "text", "Fichier"]].copy()
df_text_link = df_text_link.rename(columns={"Fichier":"url_proapp"})
df_text_link = df_text_link.rename(columns={"text":"text_short"})

In [ ]:
# We write the list of proapp links and download them (proapp are directly connected to local server, you should use the legifrance url and download.py code instead)

list_proapp = df_text_link["url_proapp"].tolist()

with open("../data/text/docx/Liste.txt", "w", encoding="utf-8") as f:
    for item in list_proapp:
        f.write("%s\n" % item)

# I used the exe file not furnished here to download the files

In [ ]:
# We convert all the files to txt and clean them

!"../data/text/docx/Recup_FichersAccord.exe"
# !python.exe "../src/download.py"
!python.exe "../src/convert_to_txt.py"
!python.exe "../src/clean_txt.py"

In [ ]:
# Now we can store the text in a csv file and match them by their code

texts = glob.glob("../data/text/txt/*.txt")

lis_of_text = []
code = []

for text in texts:
    with open(text, "r", encoding="utf-8") as f:
        lis_of_text.append(f.read())
        code.append(text[17:29])

df_long_text = pd.DataFrame({"code":code, "text":lis_of_text})

In [ ]:
# We merge the two dataframes

df_text = pd.merge(df_short_text, df_long_text, on="code", how="inner", suffixes=("_short", "_long"))
df_text["diff_length"] = df_text["text_long"].str.len() - df_text["text_short"].str.len()
df_text.to_csv('../data/processed/short_long_texts.csv', index=False)